## Titanic overlevelse

Her har vi data om passasjerer på skipet Titanic og informasjon om hvorvidt de overlevde. I denne oppgaven skal du bruke passasjerdata for å teste om det er forskjeller i overlevelsen mellom passasjergrupper og predikere om passasjeren overlevde. 

In [1]:
# imports
import numpy as np
import pandas as pd
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, log_loss

In [2]:
# les inn Titanic-data
titanic_df = pd.read_csv('data/titanic.csv')

Først endrer vi litt på data for å gjøre oppgaven enklere. Dette er ikke en god ide å gjøre når vi vil lage bestmulige modeller. 

In [3]:
# del data i mål-, og prediktorvariabler
X_cols = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
y_col = 'Survived'
reduced_df = titanic_df.loc[:, X_cols + [y_col]].dropna()
X_df = pd.get_dummies(reduced_df.loc[:, X_cols], dtype='int')
X = X_df.values
y = reduced_df[y_col].values

Del data i trenings-, validerigns-, og testdata med størrelser 70%, 15%, 15% av data. 

In [4]:
# dele data i trenings, validerings og testdata
# generer X_train, X_val, X_test, y_train, y_val, y_test

X_train, X_val, y_train, y_val = train_test_split(
    X, y, shuffle=False, test_size=0.4
)

X_val, X_test, y_val, y_test = train_test_split(
    X_val, y_val, shuffle=False, test_size=0.5
)

print(X_test)
print(y_test)

test_df = pd.DataFrame(X_test, columns=X_df.columns)
test_df[y_col] = y_test
print(test_df)

[[ 3.     33.      0.     ...  7.775   0.      1.    ]
 [ 2.      6.      0.     ... 33.      1.      0.    ]
 [ 3.     17.      1.     ...  7.0542  0.      1.    ]
 ...
 [ 1.     19.      0.     ... 30.      1.      0.    ]
 [ 1.     26.      0.     ... 30.      0.      1.    ]
 [ 3.     32.      0.     ...  7.75    0.      1.    ]]
[0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 1 0 0 1 1 0 0 1 1 0 0 0 1 0
 1 1 0 1 0 0 0 0 0 1 0 1 1 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 0 0 1 1 1 1 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 1 1 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1
 0 1 1 1 1 0 0 1 0 1 1 0 1 0 1 0 0 1 1 0 0 1 1 0 0 0 0 0 0 1 1 0]
     Pclass   Age  SibSp  Parch     Fare  Sex_female  Sex_male  Survived
0       3.0  33.0    0.0    0.0   7.7750         0.0       1.0         0
1       2.0   6.0    0.0    1.0  33.0000         1.0       0.0         1
2       3.0  17.0    1.0    0.0   7.0542         0.0       1.0         0
3       2.0  34.0    0.0    0.0  13.0000         0.0       1.0         0
4       

På treningsdata, test om menn og kvinner hadde forskjellige sannsynligheter for å overleve.

In [5]:
# test om menn og kvinner hadde forskjellig overlevelse
kvinne_survival_rate = test_df[test_df["Sex_female"] == 1]["Survived"].mean()
menn_survival_rate = test_df[test_df["Sex_male"] == 1]["Survived"].mean()

print(kvinne_survival_rate)
print(menn_survival_rate)

kjonn_test = f"Menn overlever {round(menn_survival_rate*100,2)}% mens kvinner overlever {round(kvinne_survival_rate*100,2)}% "
kjonn_test

0.7647058823529411
0.18478260869565216


'Menn overlever 18.48% mens kvinner overlever 76.47% '

På treningsdata, test om de som overlevde hadde forskjellig mean alder enn de som ikke overlevde. 

In [6]:
# test om de som overlevde hadde forskjellig mean alder enn de som ikke overlevde
survived = test_df[test_df["Survived"] == 1]["Age"].mean()
dead = test_df[test_df["Survived"] == 0]["Age"].mean()
alder_test = f"Døde average alder: {round(dead,2)} mens leve average alder {round(survived,2)}"
alder_test

'Døde average alder: 28.82 mens leve average alder 27.11'

Tren en kNN-modell med k=1, k=10 og k=50 på treningsdata. Tren også en logistisk regresjon, naive Bayes modell, tilfeldig skog og supportvektormaskin på treningsdata.

In [7]:
# tren forskjellige modeller

model_output = {}

models = {'kNN-1': KNeighborsClassifier(n_neighbors=1),
          'kNN-10': KNeighborsClassifier(n_neighbors=10),
          'kNN-50': KNeighborsClassifier(n_neighbors=1),
          "regressor":LogisticRegression(),
          "naiveBayes":MultinomialNB(),
          "forest":RandomForestClassifier(n_estimators=100),
          "supportvector":SVC()
          }

for name, model in models.items():

    output = model.fit(X_train, y_train)

    model_output[f"{name}"] = output

print(model_output)


/home/henrik/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'kNN-1': KNeighborsClassifier(n_neighbors=1), 'kNN-10': KNeighborsClassifier(n_neighbors=10), 'kNN-50': KNeighborsClassifier(n_neighbors=1), 'regressor': LogisticRegression(), 'naiveBayes': MultinomialNB(), 'forest': RandomForestClassifier(), 'supportvector': SVC()}


Sorter de ulike modellene etter nøyaktighet på valideringsdata (`sklearn.metrics.accuracy_score`).

In [8]:
# sjekk nøyaktighet for valideringsdata
accuracies = [accuracy_score(y_val,model.predict(X_val)) for model in model_output.values()]

accuracies_df = pd.DataFrame(model_output.keys(),columns=["name"])
accuracies_df["accuracy"] = accuracies
accuracies_df.sort_values(by='accuracy', ascending=False, inplace=True)
print(accuracies_df)

            name  accuracy
5         forest  0.769231
3      regressor  0.734266
1         kNN-10  0.699301
4     naiveBayes  0.692308
6  supportvector  0.678322
0          kNN-1  0.629371
2         kNN-50  0.629371


I stedet for nøyaktighet er det vanlig å bruke log-loss, som tar hensyn til en probabilistisk prediksjon. Sorter de ulike modellene etter log-loss (`sklearn.metrics.log_loss`). 

In [9]:
# sjekk log loss for valideringsdata
losses = [log_loss(y_val,model.predict(X_val)) for model in model_output.values()]
losses_df = pd.DataFrame(model_output.keys(),columns=["name"])
losses_df["loss"] = losses

losses_df.sort_values(by='loss', inplace=True)
print(losses_df)

            name       loss
5         forest   8.317766
3      regressor   9.578034
1         kNN-10  10.838301
4     naiveBayes  11.090355
6  supportvector  11.594462
0          kNN-1  13.358837
2         kNN-50  13.358837


Velg ut den beste modellen (basert på log-loss) og sjekk hvor godt den generaliserer ved å regne ut nøyaktighet og log-loss på testdata. 

In [10]:
# sjekk generaliseringsevne
best_model = RandomForestClassifier()
best_model.fit(X_train, y_train) 
generalization_accuracy = accuracy_score(y_test,best_model.predict(X_test))
generalization_logloss = log_loss(y_test,best_model.predict(X_test))
print('Nøyaktighet:', generalization_accuracy)
print('Log-loss:', generalization_logloss)

Nøyaktighet: 0.8391608391608392
Log-loss: 5.797230964683178


Gi en oppsummering over hva du har gjort og hva resultatet var. 

Lest inn titanic data, delt i test/trening/validering data

Sett litt på data om hvilket kjønn som overlevde mest (kvinner)
Sett litt på gjennomsnitt alderen til døde(litt høyere) enn de som levde

Så har jeg sett på en del forskjellige modeller, trent de på train data, funnet accuracy og log_loss score. 

Da fant jeg ut at ```RandomForestClassifier``` er den beste modellen for denne dataen.

Så skulle vi se om den faktisk kan generaliseres
og drumroll . . . det kan den!

Ganske bra og, siden Log-loss på test dataen er enda lavere en train dataen! 

Accuracy er litt høyere, men ikke mye. Så da har vi bevis at ```RandomForestClassifier``` er en god model hvis det skulle skje en titanic 2. (bank på tre)